# Decays

What we want to try to do in this notebook is to get the correct expression for the decay of the Muon without any fancy loop corrections. This exercise is one that can be found in Griffiths. We want to try to do it with SymPy.

## (#1): Libraries:

In [27]:
import sympy as sp

## (#2): Constants:

### (#2.1): Masses:

In [28]:
# LEPTONS
mass_electron_in_GeV = sp.symbols('m_e', real = True)
mass_muon_in_GeV = sp.symbols('m_mu', real = True)
mass_tau_in_GeV = sp.symbols('m_tau', real = True)

mass_electron_neutrino_in_GeV = sp.symbols('m_nu_e', real = True)
mass_muon_neutrino_in_GeV = sp.symbols('m_nu_mu', real = True)
mass_tau_neutrino_in_GeV = sp.symbols('m_nu_tau', real = True)

# QUARKS:
mass_quark_up_in_GeV = sp.symbols('m_u', real = True)
mass_quark_down_in_GeV = sp.symbols('m_d', real = True)
mass_quark_strange_in_GeV = sp.symbols('m_s', real = True)
mass_quark_charm_in_GeV = sp.symbols('m_c', real = True)
mass_quark_bottom_in_GeV = sp.symbols('m_b', real = True)
mass_quark_top_in_GeV = sp.symbols('m_t', real = True)

# GAUGE BOSONS:
mass_W_boson_in_GeV = sp.symbols('m_W', real = True)

### (#2.2): Couplings:

In [29]:
# COUPLINGS:
gW_in_GeV = sp.symbols('g_W', real = True)

### (#2.3): Symbolic Variables:

In [30]:
# VARIABLES:
m23Squared = sp.Symbol("m_23^2", real = True)
m12Squared = sp.Symbol("m_12^2", real = True)

## (#3): Two-Body Decay Function:

Two-body decays are easy because there are enough constraints in the problem that no integrations are necessary. The equation that we need to code is just

$$\Gamma \left( A \to 1 + 2 \right) = \frac{S |\mathcal{M}|^{2}}{8 \pi m_{A}^{2}} \frac{1}{2 m_{A}} \sqrt{m_{A}^{4} + m_{1}^{4} + m_{2}^{4} - 2 \left( m_{A}^{2} m_{1}^{2} + m_{A}^{2} m_{2}^{2} + m_{1}^{2} m_{2}^{2}\right)}.$$

For some more clarity, $S$ is a "spin factor" that is necessary if the two final state particles are identical (e.g. two electrons), and the fancy, huge square root term is called the $\textbf{Källén Function}$, defined as

$$\lambda(x, y, z) := x^{2} + y^{2} + z^{2} - 2 x y - 2 x z - 2 y z.$$

Of course, $\mathcal{M}$ is the amplitude of the process that we calculate within QFT (Quantum Field Theory).

### (#3.1): The Källén Function:

In [31]:
def kallen_function(x, y, z):

    try:

        # (1): Compute the polynomial immediately:
        kallen_polynomial = sp.Pow(x, 2) + sp.Pow(y, 2) + sp.Pow(z, 2) - 2 * (x * y + y * z + x * z)

        # (2): Return the polynomial:
        return kallen_polynomial
    
    except Exception as ERROR:
        print(f">Error calculating Kallen polynomial:\n {ERROR}")
        return 0.

### (#3.2): Two-Body Decay Function:

In [32]:
def decay_two_body(amplitude_squared, mass_A, mass_1,  mass_2, spin_factor: int = 1):
    
    try:

        # (1): The Spin Factor:
        spin_factor = spin_factor

        # (2): Calculate the Two-Body Decay Prefactor:
        two_body_decay_prefactor = spin_factor / (16 * sp.pi * sp.Pow(mass_A, 3))

        # (3): Evaluate the Kallen Polynomial:
        kallen_polynomial = sp.sqrt(kallen_function(sp.Pow(mass_A, 2), sp.Pow(mass_1, 2), sp.Pow(mass_2, 2)))

        # (4): Return the actual expression:
        two_body_decay_rate = two_body_decay_prefactor * kallen_polynomial * amplitude_squared
        
        # (5): Return the expression:
        return two_body_decay_rate
    
    except Exception as ERROR:
        print(f"> Error calculating two-body decay rate:\n {ERROR}")
        return 0

### (#3.3): Examples of Two-Body Decay:

#### (#3.3.1): Amplitude Squared of Unity:

In [33]:
decay_two_body(1, mass_muon_in_GeV, 0,  0, 1)


1/(16*pi*m_mu)

#### (#3.3.2): Amplitude Squared of Unity with Some Nonzero Final-State Masses:

In [34]:
decay_two_body(1, mass_muon_in_GeV, mass_tau_in_GeV, mass_electron_in_GeV)

sqrt(m_e**4 - 2*m_e**2*m_mu**2 - 2*m_e**2*m_tau**2 + m_mu**4 - 2*m_mu**2*m_tau**2 + m_tau**4)/(16*pi*m_mu**3)

## (#4): Three-Body Decay Function:

We'll now implement the "Dalitz Integration" method. This is a little fancy, and we'll document it more later.

### (#4.1): Calculate the Bounds for the $m_{23}^{2}$ Variable:

In [35]:
def calculate_m23Squared_bounds(mass_mother, mass_one, mass_two, mass_three) -> tuple:

    try:

        # (1): Compute what is called E_{2}^{*}:
        e2Star = m12Squared - sp.Pow(mass_one, 2) + sp.Pow(mass_two, 2) / (2 * sp.sqrt(m12Squared))
        
        # (2): Compute what is called E_{3}^{*}:
        e3Star = sp.Pow(mass_mother, 2) - m12Squared - sp.Pow(mass_three, 2) / (2 * sp.sqrt(m12Squared))

        # (3): Compute the fancy root quantity with E_{2}^{*}:
        root_with_e2Star = sp.sqrt(sp.Pow(e2Star, 2) - sp.Pow(mass_two, 2))

        # (4): Compute the fancy root quantity with E_{3}^{*}:
        root_with_e3Star = sp.sqrt(sp.Pow(e3Star, 2) - sp.Pow(mass_three, 2))

        # (5): Go ahead and compute m_{23}^{2}'s Lower Bound:
        m23Squared_lower_bound = sp.Pow(e2Star + e3Star, 2) - sp.Pow(root_with_e2Star + root_with_e3Star, 2)

        # (6): Go ahead and compute m_{23}^{2}'s Upper Bound:
        m23Squared_upper_bound = sp.Pow(e2Star + e3Star, 2) + sp.Pow(root_with_e2Star + root_with_e3Star, 2)

        # (7): Return the bounds as a tuple:
        return (m23Squared_lower_bound, m23Squared_upper_bound)
    
    except Exception as ERROR:
        print(f"> Error calculating three-body decay m23Squared bounds:\n {ERROR}")
        return (0, 0)

### (#4.2): Calculate the Bounds for the $m_{12}^{2}$ Variable:

In [36]:
def calculate_m12Squared_bounds(mass_mother, mass_one, mass_two, mass_three) -> tuple:
    
    try:

        # (1): Calculate m12Squared's lower bound immediately:
        m12Squared_lower_bound = sp.Pow(mass_one + mass_two, 2)

        # (2): Calculate m12Squared's upper bound immediately:
        m12Squared_upper_bound = sp.Pow(mass_mother - mass_three, 2)

        # (3): Return a tuple with the bounds:
        return (m12Squared_lower_bound, m12Squared_upper_bound)
    
    except Exception as ERROR:
        print(f"> Error calculating three-body decay m12Squared bounds:\n {ERROR}")
        return (0, 0)

### (#4.3): The Three-Body Decay Function:

In [37]:
def decay_three_body(amplitude_squared, mass_mother, mass_one, mass_two, mass_three):
    
    try:
        
        # (1): Construct the Dalitz Integral:
        integrand = amplitude_squared / (sp.Pow(2 * sp.pi, 3) * 32 * sp.Pow(mass_mother, 3))

        # (2): Obtain the bounds for m_{23}^{2}:
        m23SquaredLowerBound, m23SquaredUpperBound = calculate_m23Squared_bounds(mass_mother, mass_one, mass_two, mass_three)

        # (3): Obtain the bounds for m_{12}^{2}:
        m12SquaredLowerBound, m12SquaredUpperBound = calculate_m12Squared_bounds(mass_mother, mass_one, mass_two, mass_three)

        # (4): Attempt the integration:
        dalitz_integration = sp.integrate(integrand, (m23Squared, m23SquaredLowerBound, m23SquaredUpperBound),  (m12Squared, m12SquaredLowerBound, m12SquaredUpperBound))

        # (5): Return the result:
        return dalitz_integration

    except Exception as ERROR:
        print(f"> Error integrating the three-body Dalitz Integral:\n {ERROR}")
        return 0

### (#4.4): Examples of Three-Body Decay:

#### (#4.4.1): Amplitude Squared of Unity:

In [38]:
decay_three_body(1, mass_tau_in_GeV, 0, 0, 0)

m_tau**3/(128*pi**3)

#### (#4.4.2): Amplitude Squared of Unity with Nonzero Final-State Masses:

Actually, these three-body decay integrals are usually quite difficult if there are nonzero final-state masses. So, we'll see how our function does. Usually, we can get analytical results with *one* final-state mass. Let's see how this works:

In [39]:
decay_three_body(1, mass_tau_in_GeV, mass_electron_in_GeV, 0, 0)

Piecewise(((-m_e**2 + m_tau**2)**2/(128*pi**3*m_tau) - (-m_e**2 + m_tau**2)**2*Min(m_e**2, m_tau**2)/(128*pi**3*m_tau**3), m_e**2 < m_tau**2), (-m_e**2*(-((-m_e**2 + m_tau**2)**2 - (m_e**2 - m_tau**2)**2)/(256*pi**3*m_tau**3) + ((-m_e**2 + m_tau**2)**2 + (m_e**2 - m_tau**2)**2)/(256*pi**3*m_tau**3)) + (-((-m_e**2 + m_tau**2)**2 - (m_e**2 - m_tau**2)**2)/(256*pi**3*m_tau**3) + ((-m_e**2 + m_tau**2)**2 + (m_e**2 - m_tau**2)**2)/(256*pi**3*m_tau**3))*Min(m_e**2, m_tau**2), True))

#### (#4.4.3): The Decay of the Muon with Massless Final-State Leptons:

In [40]:
amplitude_squared_for_muon_decay = sp.Pow(gW_in_GeV, 4) * (-m23Squared + sp.Pow(mass_muon_in_GeV, 2)) * m23Squared / (2 * sp.Pow(mass_W_boson_in_GeV, 4))
amplitude_squared_for_muon_decay

g_W**4*m_23^2*(-m_23^2 + m_mu**2)/(2*m_W**4)

In [41]:
decay_three_body(amplitude_squared_for_muon_decay, mass_muon_in_GeV, 0, 0, 0)

m_mu**2*(-g_W**4*m_mu**9/(192*pi**3*m_W**4) + g_W**4*m_mu**7/(256*pi**3*m_W**4))

This is 100% incorrect. So, we need to figure out what is going on. That will be for another day.